In [28]:
import bisect
import csv

In [38]:
def cargar_datos(archivo_csv):
    """
    Cargar los datos de un archivo CSV y convertirlos en un diccionario.
    Formato esperado del CSV: id,peso,beneficio
    """
    items = {}
    with open(archivo_csv, mode='r', newline='') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            id_item = int(row[0])
            peso = int(row[1])
            beneficio = int(row[2])
            items[id_item] = (peso, beneficio)
    return items

def resolver_mochila(items, capacidad, max_iter=100, mejora_minima=1):
    """
    Resolver el problema de la mochila usando un enfoque voraz y búsqueda de vecindario.
    """
    # Generación de la solución inicial con un enfoque voraz
    item_list = [(k, v[0], v[1], v[1]/v[0]) for k, v in items.items()]
    sorted_items = sorted(item_list, key=lambda x: (-x[3], x[1]))

    selected = []
    current_weight = 0
    current_benefit = 0

    for item in sorted_items:
        if current_weight + item[1] <= capacidad:
            selected.append(item[0])
            current_weight += item[1]
            current_benefit += item[2]

    print("--- Solución inicial ---")
    print(f"Artículos seleccionados: {sorted(selected)}")
    print(f"Peso total: {current_weight}")
    print(f"Beneficio total: {current_benefit}\n")

    # Búsqueda local iterativa
    iteracion = 0
    mejora_total = 0

    while iteracion < max_iter:
        iteracion += 1
        mejora_iteracion = 0

        print(f"\n--- Iteración {iteracion} ---")
        print(f"Artículos seleccionados actualmente: {sorted(selected)}")
        print(f"Peso actual: {current_weight}, Beneficio actual: {current_benefit}")

        # Procesamiento inicial de datos
        remaining_capacity = capacidad - current_weight
        selected_data = [(s, items[s][0], items[s][1]) for s in sorted(selected)]
        unselected = [u for u in items if u not in {s[0] for s in selected_data}]
        unselected_sorted = sorted(unselected, key=lambda u: items[u][0])
        u_weights = [items[u][0] for u in unselected_sorted]
        u_benefits = [items[u][1] for u in unselected_sorted]

        print(f"Artículos no seleccionados: {unselected_sorted}")

        # Búsqueda de vecindario
        mejor_vecino = None
        mejor_beneficio = current_benefit

        for s_id, s_w, s_b in selected_data:
            max_u_weight = s_w + remaining_capacity
            max_idx = bisect.bisect_right(u_weights, max_u_weight)

            for i in range(max_idx):
                u_id = unselected_sorted[i]
                u_w = u_weights[i]
                u_b = u_benefits[i]
                nuevo_peso = current_weight - s_w + u_w
                nuevo_beneficio = current_benefit - s_b + u_b

                # Verificar mejora
                if nuevo_beneficio > mejor_beneficio:
                    mejor_beneficio = nuevo_beneficio
                    mejor_vecino = (u_id, s_id, nuevo_peso, nuevo_beneficio)
                    print(f"Intercambio propuesto: Entra {u_id} (Peso: {u_w}, Beneficio: {u_b}), "
                          f"Sale {s_id} (Peso: {s_w}, Beneficio: {s_b})")
                    print(f"Nuevo peso: {nuevo_peso}, Nuevo beneficio: {nuevo_beneficio}")

        # Aplicar la mejor mejora encontrada
        if mejor_vecino:
            u_id, s_id, nuevo_peso, nuevo_beneficio = mejor_vecino
            selected.remove(s_id)
            selected.append(u_id)
            current_weight = nuevo_peso
            mejora_iteracion = nuevo_beneficio - current_benefit
            current_benefit = nuevo_beneficio
            mejora_total += mejora_iteracion
            print(f"\nMejor intercambio aplicado: Entra {u_id}, Sale {s_id}")
            print(f"Peso actualizado: {current_weight}, Beneficio actualizado: {current_benefit}")
            print(f"Mejora en esta iteración: {mejora_iteracion}")
        else:
            print("\nNo se encontraron intercambios que mejoren el beneficio.")
            break  # Detener si no hay mejora

        # Detener si la mejora es menor que el umbral
        if mejora_iteracion < mejora_minima:
            print(f"\nLa mejora en esta iteración ({mejora_iteracion}) es menor que el umbral mínimo ({mejora_minima}).")
            break

    # Resultados finales
    print("\n--- Resultados finales ---")
    print(f"Solución final: {sorted(selected)}")
    print(f"Peso total: {current_weight}")
    print(f"Beneficio total: {current_benefit}")
    print(f"Mejora total: {mejora_total}")
    print(f"Iteraciones realizadas: {iteracion}")

In [39]:
# Ejemplo de uso
if __name__ == "__main__":
    archivo_csv = "datos_mochila_2.csv"  # Ruta del archivo CSV
    capacidad = 400  # Capacidad de la mochila
    max_iter = 50  # Número máximo de iteraciones
    mejora_minima = 1  # Umbral de mejora mínima

    # Cargar datos desde el archivo CSV
    items = cargar_datos(archivo_csv)

    # Resolver el problema de la mochila
    resolver_mochila(items, capacidad, max_iter, mejora_minima)

--- Solución inicial ---
Artículos seleccionados: [1, 2, 4, 7, 9, 10, 11, 12, 14, 15]
Peso total: 355
Beneficio total: 2670


--- Iteración 1 ---
Artículos seleccionados actualmente: [1, 2, 4, 7, 9, 10, 11, 12, 14, 15]
Peso actual: 355, Beneficio actual: 2670
Artículos no seleccionados: [5, 13, 8, 3, 6]
Intercambio propuesto: Entra 5 (Peso: 80, Beneficio: 390), Sale 2 (Peso: 50, Beneficio: 320)
Nuevo peso: 385, Nuevo beneficio: 2740
Intercambio propuesto: Entra 13 (Peso: 90, Beneficio: 420), Sale 2 (Peso: 50, Beneficio: 320)
Nuevo peso: 395, Nuevo beneficio: 2770
Intercambio propuesto: Entra 5 (Peso: 80, Beneficio: 390), Sale 7 (Peso: 40, Beneficio: 280)
Nuevo peso: 395, Nuevo beneficio: 2780

Mejor intercambio aplicado: Entra 5, Sale 7
Peso actualizado: 395, Beneficio actualizado: 2780
Mejora en esta iteración: 110

--- Iteración 2 ---
Artículos seleccionados actualmente: [1, 2, 4, 5, 9, 10, 11, 12, 14, 15]
Peso actual: 395, Beneficio actual: 2780
Artículos no seleccionados: [7, 13, 8